In [3]:
import json
import requests
import re
from pyquery import PyQuery as pq

In [4]:
with open('links.json', 'r') as input:
    links=input.read()

urls = json.loads(links)

In [5]:
dataSet=[]
headers={"accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
"accept-encoding":"gzip, deflate, br",
"accept-language":"en-US,en;q=0.9",
"cache-control":"max-age=0",
"sec-ch-ua":'"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
"sec-ch-ua-mobile":"?0",
"sec-ch-ua-platform":'"Windows"',
"sec-fetch-dest":"document",
"sec-fetch-mode":"navigate",
"sec-fetch-site":"cross-site",
"sec-fetch-user":"?1",
"upgrade-insecure-requests":"1",
#"cookie":"px_init=0; header_signin_prompt=1; cors_js=1; _gid=GA1.2.2022774677.1682988212; _gat=1; BJS=-; lastSeen=0; bkng_sso_session=e30; OptanonConsent=implicitConsentCountry=nonGDPR&implicitConsentDate=1682988212603; _gcl_au=1.1.823967361.1682988213; pxcts=5a9558c0-e882-11ed-be3a-4c6777456c70; _pxvid=5a954d04-e882-11ed-be3a-4c6777456c70; _pxff_ddtc=1; _ga_FPD6YLJCJ7=GS1.1.1682988213.1.0.1682988213.0.0.0; _ga=GA1.1.1069262369.1682988212; bkng_sso_ses=eyJib29raW5nX2dsb2JhbCI6W3siYSI6MSwiaCI6ImdmK3pYNG1ESGJ5dVpKZlJrTHoyVGRZOGtFVDJLNXFrRDJVNjQxNlhyR1UifV19; _uetsid=5e2977d0e88211eda922fb0b8ac06362; _uetvid=5e29dd10e88211ed8f97fb8c0ab63ba4; _px3=cc88c9534df149a83673e83a7ef2ed39b4485d1c08979c584899c257555be828:CCkkxfDtJuACv9tW/AmFO24mP27jGIarZNZgH8Rm+K8INArIwDaPXlcGWkQBK7O1SJVsDPnqZ0luMsMsj5CT/A==:1000:Oh0Sg0JRhChQUnKu0i2yCcwTcksO3sq20mp3nVnRF2Cu/6JxmQn5dI9Gh6liztA3DMiLGa6MIlvHE74IPSmvdKhLQVWBsBnaf0HoBf7kNZF1KEKi/dKxBIscbjF1qYQC5HGD1Cs/BevVYHyl1wrwOTEkdvPl1MvKJ2CT/VAGiwWpBSAw5W4o4E3tujFFlcEHHxSzZ3nBPJWxcSAbmQNqpw==; _pxde=70fa30dcc820f21e36c0c0d24d1bde89633f63042d60115cf4f238c148302c93:eyJ0aW1lc3RhbXAiOjE2ODI5ODgyMDkyNzIsImZfa2IiOjAsImlwY19pZCI6W119; _pxhd=%2FXU46Mfq140-gKOEaGV8BQKAYyysTQGd-pskeUFZ2c7OU-9yeUT4nDoQpgHYKm7PYt8nNKsoGagSyu6WzfTRGQ%3D%3D%3AaUCl6pEG7bdTEK%2Fb%2F77uNOHSOKdwa9FKwdnnA5-MnaIJmgn33Zli-wgS6YdmkwPCHmUwXDwYFsg1jOrDxcs%2FQg10mV43DlRb%2F4auwBL2XnM%3D; px_init=0; bkng=11UmFuZG9tSVYkc2RlIyh9Yaa29%2F3xUOLblgO%2Fz4BDP5urnT05osPuQQ5A7de3SZq98vsGkz1CHzjulB4HPR7G%2FGfH2sT3bjeFU2dWz6OyTeGznnesC3UQ1SUyZFUx1as1Gc1xNhj%2Bhosu7yM0GzH78WBd2MtsVq73tzoCrwQk%2BY37I0HWZso0EfMdA2r35yevRWKtGshUEcU%3D",
#"referer":"https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html",
"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

In [6]:
requests.get("https://www.booking.com")
for url in urls["urls"]:
    baseUrl=url
    offset=25
    print(f"URL : {baseUrl}")
    page=1
    nextPage = True
    while nextPage: 
        # if page>1:
        response = requests.get(url).text
        html = pq(response).find('body')
        print(f"Page {page}")
        if page == 1:
            # response = requests.get(url).text
            #head = pq(response).find('head')
            # html = pq(response).find('body')
            # with open("body.txt","w+") as f:
                # f.write(response)
                
            # print(response)
            # break
            sid=html.find('form#frm:first input[name="sid"]').attr('value')
            label=html.find('form#frm:first input[name="label"]').attr('value')
            name = html.find('meta[itemprop="name"]').attr('content')
            total = html.find('.hotel_rank').attr('data-total-hotels')
            ranking = html.find('.hotel_rank').attr('data-ranking-as')
            reviewCity = html.find('.hotel_rank').attr('data-reviews-city')
            
            reviewScore = html.find('.review-score-badge:first').text()            
            if reviewScore is None:
                reviewScore = html.find('meta[itemprop="ratingValue"]').attr('content')
            
            reviewBasedOn = html.find('.review_list_score_count').text()
            if reviewBasedOn is None:
                reviewBasedOn = html.find('meta[itemprop="ratingCount"]').attr('content')            
            rank = f"{name} -- {ranking} out of {total} from {reviewCity} -- Review Score:{reviewScore}, {reviewBasedOn}"    
            print(rank)
           
            
            listScores = html.find('#review_list_score_breakdown li')  
            scores={}
            for list in listScores:
                li = pq(list)
                key = li.find('.review_score_name').text()
                if key is not None:
                    value = li.find('.review_score_value').text()
                    key = re.sub(r"\s",'_',key.lower().strip())
                    scores[key]=value  
            hotelData={'url':baseUrl,'name':name,'ranking':ranking,'total':total,'reviewCity':reviewCity,
                           'reviewScore':reviewScore,'reviewBasedOn':reviewBasedOn,'scores':scores}
        #check next Page
        pageNext = html.find('p.review_next_page a#review_next_page_link').attr('href')
        if pageNext:             
            page=page+1
            offset=offset+25
            # urlStr=f"&customer_type=total&hp_nav=0&old_page=0&order=featuredreviews&page={page}&r_lang=en&rows={offset}&"            
            url="https://www.booking.com"+pageNext
            # if sid is not None and label is not None:
                # url="https://www.booking.com?label="+label+"&sid="+sid+urlStr
            print(f"NEXT: {page} {offset} {url}")
        else:                    
            nextPage=False
        reviews = html.find('ul.review_list li.review_item')
        for review in reviews:
            review = pq(review)
            published = review.find('p.review_item_date').text()
            published = re.sub(r"Reviewed:",'',published).strip() #Reviewed:
            author = review.find('.reviewer_name').text()
            country = review.find('[itemprop="nationality"] [itemprop="name"]').text()
            author_reviewcount = review.find('.review_item_user_review_count').text() 
            author_reviewcount = re.sub(r"reviews",'',author_reviewcount).strip() #reviews
            rating = review.find('.review-score-badge').text()
            if rating is None:
                rating = review.find('[itemprop="reviewRating"] meta[itemprop="ratingValue"]').attr('content')
            title = review.find('.review_item_header_content span').text()
            tags=[]
            for tag in review.find('ul.review_item_info_tags li.review_info_tag'):
                tag = pq(tag)
                tagText = re.sub(r"^(\W+)\s+",'',tag.text()) 
                tagText = re.sub(r"\&bull\;",'',tagText)
                tagText = re.sub(r"\\u2022",'',tagText)
                tagText = re.sub(r"^(\W+)\s+",'',tagText.strip()) 
                # tagText = str.replace('• ','',tagText).strip()  
                tags.append(tagText)            
            positive = review.find('.review_pos [itemprop="reviewBody"]').text()
            negative = review.find('.review_neg [itemprop="reviewBody"]').text()
            stay = review.find('.review_staydate').text()
            stay = re.sub(r"Stayed\sin",'',stay).strip()
            
            
            dataSet.append(
                {'hotel':hotelData,'reviews':{'title':title,'published':published,'author':author,'country':country,
                'author_reviewcount':author_reviewcount,'rating':rating,'tags':' | '.join(tags),'positive':positive,
                'negative':negative,'stayDate':stay}})
            

URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html
Page 1
Miller & Carter Heaton Chapel by Innkeeper's Collection -- 48 out of 109 from Manchester -- Review Score:8.3, Based on 249 hotel reviews
NEXT: 2 50 https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html?page=2&
Page 2
NEXT: 3 75 https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html?page=3&
Page 3
NEXT: 4 100 https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-stockport-heaton-chapel.en-gb.html?page=4&
Page 4
URL : https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-chester-christleton.en-gb.html
Page 1
Innkeeper's Collection Chester, Christleton -- 9 out of 31 from Chester -- Review Score:8.6, Based on 1227 hotel reviews
NEXT: 2 50 https://www.booking.com/reviews/gb/hotel/innkeeper-s-lodge-chester-christleton.en-gb.html?page=2&
Page 2
NEXT: 3 75 https://www.booking.com/reviews/gb/hotel/inn

In [38]:
# dataSet

In [7]:
import json

with open("reviews.json", "w") as file:
    json.dump(dataSet, file, indent=4, sort_keys=False)